In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math
keras = tf.keras


2023-03-14 12:40:44.609475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
test = tf.data.Dataset.from_tensor_slices([1, 2, 3])

In [51]:


import sys
sys.path.append("..")

from mechanism.mechanized_sequential import MechanizedSequential
from mechanism.mechanized_sequential import Mechanism



In [68]:
Q_MEAN = 0.5
EPOCH = 2
POPULATION_SIZE = 10000
TRAIN_DIM = 100
STEP = 1

In [59]:

import numpy as np

def initialize_with_str_seed(init_str):
    """
    Initializes random number generator with seed corresponding to given input string init_str.
    :param init_str: Initialization string according to which seed will be computed. Seed is the sum of the ASCII
                     values of each character in init_str.
    """
    rnd_val = 0
    if init_str:
        for c in init_str:
            rnd_val += ord(c)
    np.random.seed(rnd_val)

def gen_data(n, d, seed = None):
    if seed:
        initialize_with_str_seed(seed)
    p = (1.0 + np.sqrt(max(2 * Q_MEAN - 1, 1 - 2 * Q_MEAN))) / 2 
    data = np.random.choice([-1, 1], (n, d), p=[1 -p, p])
    data_y = np.random.choice([0, 1], n, p=[1 -p, p])
    return data, data_y

def gen_valid(n, d, seed = None):
    if seed:
        initialize_with_str_seed(seed)
    
    n = int(n/10)
    
    p = (1.0 + np.sqrt(max(2 * Q_MEAN - 1, 1 - 2 * Q_MEAN))) / 2 
    data = np.random.choice([-1, 1], (n, d), p=[1 -p, p])
    data_y = np.random.choice([0, 1], n, p=[1 -p, p])
    return data, data_y



In [78]:
import math

x_train, y_train = gen_data(POPULATION_SIZE, TRAIN_DIM)
x_valid, y_valid = gen_data(int(POPULATION_SIZE/500), TRAIN_DIM)


In [92]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [102]:

def compile_and_fit_model(model, train_set, epoch_num = EPOCH, eager = False):

    
    lr_schedule = keras.callbacks.LearningRateScheduler(
        lambda epoch: 1e-8 * 10**(epoch / 20))
    optimizer = keras.optimizers.SGD(lr=1e-8, momentum=0.9)
    model.compile(loss=keras.losses.Huber(),
                optimizer=optimizer,
                metrics=["mae"],
                run_eagerly = eager)

    history = model.fit(train_set, epochs = epoch_num, callbacks=[lr_schedule])
    return history

def eval_model(train_size, epoch_num = EPOCH, mechanism = Mechanism(Mechanism.MechanismType.NONE)):
    ''' Compile and fit the empirical model as baseline'''
    model = MechanizedSequential([
        keras.layers.Conv1D(filters = 32,
                            kernel_size = 5,
                            strides = 1, 
                            padding = "causal",
                            activation = "relu",
                            input_shape = [None, 1]),
        # keras.layers.LSTM(32, return_sequences = True),
        # keras.layers.LSTM(32, return_sequences=True),
        keras.layers.Dense(1),
        keras.layers.Lambda(lambda x: x * 200)
    ])    
    batch_size = math.floor(train_size/STEP)
    train_set = tf.data.Dataset.from_tensor_slices((x_train[:train_size], y_train[:train_size])).batch(batch_size)


    if mechanism:
        model.choose_mech(mechanism)
        history = compile_and_fit_model(model, train_set, epoch_num, True)
    else:
        history = compile_and_fit_model(model, train_set, epoch_num, False)


    ''' Validate the result'''
    # x_valid, y_valid = gen_valid(POPULATION_SIZE, TRAIN_DIM)
    # model.choose_mech(None)
    predict = model.predict(x_valid)
    
    error = generalization_error(y_valid, predict)
    return history, predict, error


def generalization_error(true_val, predict_val):
    error = keras.metrics.RootMeanSquaredError()
    error.update_state(true_val, predict_val)
    return error.result().numpy()


def eval_multiple_rounds(train_size, stepped_epoch_num, mechanism = Mechanism(Mechanism.MechanismType.NONE)):
    history_list, predict_list, generalization_error_list = [], [], []
    for r in stepped_epoch_num:
        history, predict, error = eval_model(train_size = train_size, epoch_num = r, mechanism = mechanism)
        history_list.append(history)
        predict_list.append(predict)
        generalization_error_list.append(error) 

    return history_list, predict_list, generalization_error_list

def eval_const_rounds(stepped_train_size, epoch_num = EPOCH, mechanism = Mechanism(Mechanism.MechanismType.NONE)):
    history_list, predict_list, generalization_error_list = [], [], []
    for n in stepped_train_size:
        history, predict, error = eval_model(train_size = n, epoch_num = epoch_num, mechanism = mechanism)
        history_list.append(history)
        predict_list.append(predict)
        generalization_error_list.append(error) 

    return history_list, predict_list, generalization_error_list




# Evaluate the Model

In [71]:


'''
Plot the Comparison of the Three Model
'''
def plot_error(rounds, generalization_error, mechanism):
    plt.plot(rounds, generalization_error, label = mechanism)
    plt.xlabel("Queries")
    plt.ylabel("RMSE (Generalization Error) for adaptive queries")
    plt.legend()
    plt.grid()


' eval the empirical model as baseline'

In [110]:
max_query_num = 200

stepped_query_num = range(math.floor(max_query_num/2), max_query_num, 20)

In [ ]:
baseline_generalization_error_list = eval_const_rounds(stepped_query_num)[2]

In [122]:
print(baseline_generalization_error_list)

[[53.79278, 40.637043, 47.17631, 67.091385, 52.12748], [37.245102, 42.420113, 51.868324, 45.043446, 36.067776], [90.73652, 43.31463, 45.375214, 34.43256, 44.133877], [54.023632, 37.394985, 59.68137, 61.353947, 49.137417], [41.985214, 37.61297, 51.215202, 33.56925, 57.74051], [30.129105, 41.899986, 43.523647, 46.9222, 49.066086], [61.431465, 25.781137, 67.34361, 51.353867, 39.698456], [53.02296, 57.734634, 36.514366, 55.816998, 37.11971], [39.366055, 44.84889, 59.51758, 40.04781, 59.55494], [56.89471, 53.45872, 22.450909, 46.027973, 44.207947], [49.178284, 59.935642, 55.08327, 55.52672, 37.82615], [37.611813, 47.93302, 43.04385, 46.061237, 32.50882], [52.064644, 43.579296, 38.16288, 37.108517, 84.59685], [43.52374, 33.783665, 53.79379, 37.97431, 44.404274], [33.90118, 46.84127, 41.82737, 30.031021, 53.407436], [75.37575, 57.30616, 48.73929, 61.507656, 58.317314], [29.11169, 63.57325, 35.23529, 42.81679, 63.158066], [48.709724, 46.680706, 46.668438, 51.205715, 43.886143], [47.255077, 58.

In [121]:
print(np.mean(baseline_generalization_error_list, axis = 0))

[47.34263  46.127934 49.156864 46.69148  46.0877  ]


In [120]:
''' eval the empirical model as baseline'''
baseline_generalization_error_list = [eval_const_rounds(stepped_query_num)[2] for _ in range(100)]
print(baseline_generalization_error_list)
"""
[13.719473]
[10.51995]
"""

Epoch 1/2
1/1 [==============================] - 0s 385ms/step - loss: 42.1307 - mae: 42.6267 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 29ms/step


Epoch 1/2
1/1 [==============================] - 1s 588ms/step - loss: 32.3605 - mae: 32.8605 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 39ms/step


Epoch 1/2
1/1 [==============================] - 1s 689ms/step - loss: 36.3285 - mae: 36.8285 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 27ms/step


Epoch 1/2
1/1 [==============================] - 1s 1s/step - loss: 54.2641 - mae: 54.7641 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 30ms/step


Epoch 1/2
1/1 [==============================] - 1s 996ms/step - loss: 42.1474 - mae: 42.6390 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 23ms/step


...

Epoch 1/2
1/1 [==============================] - 0s 437ms/step - loss: 43.5516 - mae: 44.0516 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 24ms/step


Epoch 1/2
1/1 [==============================] - 1s 576ms/step - loss: 36.7038 - mae: 37.2038 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 25ms/step


Epoch 1/2
1/1 [==============================] - 1s 751ms/step - loss: 49.8785 - mae: 50.3785 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 21ms/step


Epoch 1/2
1/1 [==============================] - 1s 949ms/step - loss: 35.4176 - mae: 35.9136 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 22ms/step
[[53.79278, 40.637043, 47.17631, 67.091385, 52.12748], [37.245102, 42.420113, 51.868324, 45.043446, 36.067776], [90.73652, 43.31463, 45.375214, 34.43256, 44.133877], [54.023632, 37.394985, 59.68137, 61.353947, 49.137417], [41.985214, 37.61297, 51.215202, 33.56925, 57.74051], [30.129105, 41.899986, 43.523647, 46.9222, 49.066086], [61.431465, 25.781137, 67.34361, 51.353867, 39.698456], [53.02296, 57.734634, 36.514366, 55.816998, 37.11971], [39.366055, 44.84889, 59.51758, 40.04781, 59.55494], [56.89471, 53.45872, 22.450909, 46.027973, 44.207947], [49.178284, 59.935642, 55.08327, 55.52672, 37.82615], [37.611813, 47.93302, 43.04385, 46.061237, 32.50882], [52.064644, 43.579296, 38.16288, 37.108517, 84.59685], [43.52374, 33.783665, 53.79379, 37.97431, 44.404274], [33.90118, 46.84127, 41.82737, 30.031021, 53.407436], [75.37

'\n[13.719473]\n[10.51995]\n'

In [119]:
print(np.mean(gaussian_generalization_error_list, axis = 0))

[47.40192  47.06129  44.86319  46.530285 45.77672 ]


In [ ]:
gaussian_generalization_error_list = eval_const_rounds(stepped_query_num, mechanism = Mechanism(Mechanism.MechanismType.GAUSSIAN, sigma = 0.03))[2]

In [117]:
gaussian_generalization_error_list = [eval_const_rounds(stepped_query_num, mechanism = Mechanism(Mechanism.MechanismType.GAUSSIAN, sigma = 0.03))[2] for _ in range(100)]

Epoch 1/2
1/1 [==============================] - 0s 328ms/step - loss: 27.7596 - mae: 28.2591 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 26ms/step


Epoch 1/2
1/1 [==============================] - 0s 432ms/step - loss: 31.3891 - mae: 31.8852 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 65ms/step


Epoch 1/2
1/1 [==============================] - 1s 549ms/step - loss: 47.5519 - mae: 48.0519 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 32ms/step


Epoch 1/2
1/1 [==============================] - 1s 736ms/step - loss: 41.6030 - mae: 42.1030 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 22ms/step


Epoch 1/2
1/1 [==============================] - 1s 976ms/step - loss: 29.6523 - mae: 30.1523 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 23ms/step


...

Epoch 1/2
1/1 [==============================] - 0s 386ms/step - loss: 61.7277 - mae: 62.2272 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 34ms/step


Epoch 1/2
1/1 [==============================] - 1s 642ms/step - loss: 21.7292 - mae: 22.2210 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 30ms/step


Epoch 1/2
1/1 [==============================] - 1s 672ms/step - loss: 30.7892 - mae: 31.2892 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 42ms/step


Epoch 1/2
1/1 [==============================] - 1s 962ms/step - loss: 45.9493 - mae: 46.4493 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 31ms/step


Epoch 1/2
1/1 [==============================] - 1s 1s/step - loss: 14.8538 - mae: 15.3406 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 0s 28ms/step


In [100]:
print(np.mean(gaussian_generalization_error_list, axis = 0))

[11.00834]


In [86]:
''' Compile and fit the gaussian model'''
gaussian_history_list, gaussian_predict_list, gaussian_generalization_error_list = eval_const_rounds(stepped_query_num, mechanism = Mechanism(Mechanism.MechanismType.GAUSSIAN, sigma = 0.03))

Epoch 1/2
1/1 [==============================] - 4s 4s/step - loss: 13.1036 - mae: 13.6031 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 2s 2s/step


In [98]:
print(gaussian_generalization_error_list)
"""
[6.6889744]

"""

[[17.087473], [24.96883], [9.488181], [3.6728349], [15.929171], [29.725958], [4.1225915], [4.452208], [4.0646253], [12.599819], [10.871292], [6.1323094], [14.285969], [7.289968], [3.6529484], [6.2008886], [13.612192], [4.364443], [9.633908], [11.446422], [3.2145362], [11.753226], [10.50949], [6.9160337], [10.524138], [21.344978], [10.21301], [2.707585], [6.8157225], [6.893182], [16.439095], [8.153187], [7.1377606], [4.5811906], [18.894405], [11.908693], [18.596394], [15.135308], [4.631251], [14.666032], [14.975883], [20.612736], [7.6491613], [17.403416], [22.568861], [9.086757], [15.775564], [4.1009774], [10.213425], [5.671192], [16.830591], [19.125885], [5.210912], [3.4540749], [13.713624], [6.91902], [19.624445], [6.451054], [11.553105], [17.369604], [17.658655], [7.941073], [3.7584302], [9.907522], [9.760618], [4.9454274], [10.528169], [4.856306], [10.611782], [9.454437], [8.205906], [4.4913845], [4.628914], [9.275808], [8.715808], [15.782827], [17.650242], [15.39209], [4.234276], [

'\n[6.6889744]\n\n'

In [88]:

''' Compile and fit the threshold out model'''
threshold_history_list, threshold_predict_list, threshold_generalization_error_list = eval_const_rounds(stepped_train_size = stepped_query_num, mechanism = Mechanism(Mechanism.MechanismType.THRESHOLD, sigma = 0.1, hold_frac = 0.4, threshold = 0.5))
print(threshold_generalization_error_list)
"""
[3.270109] 
"""


Epoch 1/2
1/1 [==============================] - 4s 4s/step - loss: 8.1451 - mae: 8.6350 - lr: 1.0000e-08
Epoch 2/2
1/1 [==============================] - 2s 2s/step
[10.2933655]


'\n[3.270109] \n'

In [ ]:

""" plot the generalization error """
plt.figure()
plot_error(stepped_query_num, baseline_generalization_error_list, "Baseline")
plot_error(stepped_query_num, gaussian_generalization_error_list, "Gaussian")
plot_error(stepped_query_num, threshold_generalization_error_list, "Threshold - AdaptFun")
plt.show()

In [74]:
max_query_num = 10
max_rounds = 20
stepped_rounds = range(math.floor(max_rounds/2), max_rounds, 10)

In [75]:
''' eval the empirical model as baseline'''
baselin_history_list, baseline_predict_list, baseline_generalization_error_list = eval_multiple_rounds(max_query_num, stepped_rounds)
print(baseline_generalization_error_list)

Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 7.5778 - mae: 8.0772 - lr: 1.0000e-08
Epoch 2/10
1/1 [==============================] - 4s 4s/step - loss: 7.5762 - mae: 8.0756 - lr: 1.1220e-08
Epoch 3/10
1/1 [==============================] - 4s 4s/step - loss: 7.5730 - mae: 8.0724 - lr: 1.2589e-08
Epoch 4/10
1/1 [==============================] - 4s 4s/step - loss: 7.5680 - mae: 8.0674 - lr: 1.4125e-08
Epoch 5/10
1/1 [==============================] - 4s 4s/step - loss: 7.5613 - mae: 8.0607 - lr: 1.5849e-08
Epoch 6/10
1/1 [==============================] - 4s 4s/step - loss: 7.5527 - mae: 8.0521 - lr: 1.7783e-08
Epoch 7/10
1/1 [==============================] - 4s 4s/step - loss: 7.5422 - mae: 8.0415 - lr: 1.9953e-08
Epoch 8/10
1/1 [==============================] - 4s 4s/step - loss: 7.5294 - mae: 8.0288 - lr: 2.2387e-08
Epoch 9/10
1/1 [==============================] - 4s 4s/step - loss: 7.5144 - mae: 8.0138 - lr: 2.5119e-08
Epoch 10/10
32/32 [==================

In [79]:
''' Compile and fit the gaussian model'''
gaussian_history_list, gaussian_predict_list, gaussian_generalization_error_list = eval_multiple_rounds(max_query_num, stepped_rounds, mechanism = Mechanism(Mechanism.MechanismType.GAUSSIAN, sigma = 0.03))

Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 10.4830 - mae: 10.9826 - lr: 1.0000e-08
Epoch 2/10
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.10/site-packages/keras/backend.py", line 5132, in <genexpr>
    output_ta_t = tuple(  File "/usr/local/lib/python3.10/site-packages/tensorflow/python/util/tf_should_use.py", line 243, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.10/site-packages/keras/backend.py", line 5132, in <genexpr>
    output_ta_t = tuple(  File "/usr/local/lib/python3.10/site-packages/tensorflow/python/util/tf_should_use.py", line 243, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


1/1 [==============================] - 4s 4s/step - loss: 10.4815 - mae: 10.9812 - lr: 1.1220e-08
Epoch 3/10
1/1 [==============================] - 4s 4s/step - loss: 10.4785 - mae: 10.9782 - lr: 1.2589e-08
Epoch 4/10
1/1 [==============================] - 4s 4s/step - loss: 10.4740 - mae: 10.9736 - lr: 1.4125e-08
Epoch 5/10
1/1 [==============================] - 4s 4s/step - loss: 10.4678 - mae: 10.9675 - lr: 1.5849e-08
Epoch 6/10
1/1 [==============================] - 4s 4s/step - loss: 10.4600 - mae: 10.9596 - lr: 1.7783e-08
Epoch 7/10
1/1 [==============================] - 5s 5s/step - loss: 10.4502 - mae: 10.9499 - lr: 1.9953e-08
Epoch 8/10
1/1 [==============================] - 4s 4s/step - loss: 10.4386 - mae: 10.9382 - lr: 2.2387e-08
Epoch 9/10
1/1 [==============================] - 4s 4s/step - loss: 10.4247 - mae: 10.9244 - lr: 2.5119e-08
Epoch 10/10
1/1 [==============================] - 2s 2s/step


In [81]:
print(gaussian_generalization_error_list)

[11.296159]


In [ ]:

''' Compile and fit the threshold out model'''
threshold_history_list, threshold_predict_list, threshold_generalization_error_list = eval_multiple_rounds(max_query_num, stepped_rounds, mechanism = Mechanism(Mechanism.MechanismType.THRESHOLD, sigma = 0.1, hold_frac = 0.4, threshold = 0.5))
print(threshold_generalization_error_list)

In [ ]:



# threshold_generalization_error_list = [15.81994, 19.699022, 23.081583, 25.196918, 26.020424]

""" plot the generalization error """
plt.figure()
plot_error(stepped_query_num, baseline_generalization_error_list, "Baseline")
plot_error(stepped_query_num, gaussian_generalization_error_list, "Gaussian")
plot_error(stepped_query_num, threshold_generalization_error_list, "Threshold - AdaptFun")
plt.show()